In [1]:
import pandas as pd
from pathlib import Path
from labbench import stopwatch
from importlib import reload
import seamf, ziparchive
import zipfile
ziparchive = reload(ziparchive)
from ziparchive import read_seamf_zipfile, MultiProcessingZipFile, concat_dicts

data_path = Path('data')/'NIT-2022-12-13.zip'

In order to run this file, you'll need to download a zip file archive from sharepoint.

### Basic aggregation

In [2]:
import zipfile 
with stopwatch(), zipfile.ZipFile(data_path) as zfile:
    filelist = [n for n in zfile.namelist() if n.endswith('.sigmf')][:1000]
    dfs = read_seamf_zipfile(zfile, allowlist=filelist)

 INFO   2023-02-21 16:54:39.035 • labbench: 13.503 s elapsed


In [3]:
dfs['pfp']

Frame time elapsed (s)                                           0.000000  \
datetime                frequency    capture_statistic detector             
2022-10-26 20:47:37.094 3.555000e+09 min               rms       -88.4375   
                                     max               rms       -79.2500   
                                     mean              rms       -84.3125   
                                     min               peak      -78.2500   
                                     max               peak      -69.8750   
...                                                                   ...   
2022-11-06 13:31:27.913 3.695000e+09 max               rms       -91.5625   
                                     mean              rms       -93.3750   
                                     min               peak      -84.4375   
                                     max               peak      -76.4375   
                                     mean              peak      -79.8750   

Frame time elapsed (s)                                           0.000018  \
datetime                frequency    capture_statistic detector             
2022-10-26 20:47:37.094 3.555000e+09 min               rms       -89.2500   
                                     max               rms       -81.1250   
                                     mean              rms       -86.4375   
                                     min               peak      -77.9375   
                                     max               peak      -70.1875   
...                                                                   ...   
2022-11-06 13:31:27.913 3.695000e+09 max               rms       -94.3750   
                                     mean              rms       -95.5625   
                                     min               peak      -86.8750   
                                     max               peak      -79.0000   
                                     mean              peak      -82.8125   

Frame time elapsed (s)                                           0.000036  \
datetime                frequency    capture_statistic detector             
2022-10-26 20:47:37.094 3.555000e+09 min               rms       -90.9375   
                                     max               rms       -85.7500   
                                     mean              rms       -88.0000   
                                     min               peak      -80.9375   
                                     max               peak      -69.8125   
...                                                                   ...   
2022-11-06 13:31:27.913 3.695000e+09 max               rms       -95.9375   
                                     mean              rms       -96.9375   
                                     min               peak      -89.7500   
                                     max               peak      -81.9375   
                                     mean              peak      -85.3750   

Frame time elapsed (s)                                           0.000054  \
datetime                frequency    capture_statistic detector             
2022-10-26 20:47:37.094 3.555000e+09 min               rms       -93.8750   
                                     max               rms       -87.8125   
                                     mean              rms       -91.9375   
                                     min               peak      -83.1875   
                                     max               peak      -74.1250   
...                                                                   ...   
2022-11-06 13:31:27.913 3.695000e+09 max               rms       -97.1250   
                                     mean              rms       -97.9375   
                                     min               peak      -90.7500   
                                     max               peak      -83.5625   
                                     mean              peak      -87.2

### Multiprocessing aggregation (dask.bag)

In [4]:
ziparchive = reload(ziparchive)
from ziparchive import read_seamf_zipfile, MultiProcessingZipFile, concat_dicts
from dask import bag

def zipfile_bag(data_path, limit_count=None, partition_size=30):
    zfile = MultiProcessingZipFile(data_path)
    filelist = [n for n in zfile.namelist() if n.endswith('.sigmf')][:limit_count]
    return (
        bag
        .from_sequence(filelist, partition_size=partition_size)
        .map_partitions(lambda files: [read_seamf_zipfile(zfile, allowlist=files)])
    )

with stopwatch():
    dfs = (
        zipfile_bag(data_path, limit_count=1000, partition_size=30)
        .fold(concat_dicts, concat_dicts, split_every=10)
        .compute(scheduler='processes')
    )

 INFO   2023-02-17 13:00:21.053 • labbench: 11.037 s elapsed


### Multiprocessing aggregation (dask.dataframe via from_map)
This is shown aggregating a single trace type (pfp) since it loads a single dataframe.

In [15]:
import ziparchive
ziparchive = reload(ziparchive)
from ziparchive import read_seamf_zipfile, _read_seamf_zipfile_divisions, MultiProcessingZipFile, concat_dicts
import numpy as np
from seamf import iso_to_datetime
from dask import dataframe
import dask

def select(data: dict, key: str) -> pd.DataFrame:
    return data[key]

partition_size = 100
zfile = MultiProcessingZipFile(data_path)
filelist = [n for n in zfile.namelist() if n.endswith('.sigmf')][:]
file_blocks = np.split(filelist, range(0,len(filelist), partition_size))[1:]

def read_for_export(files):
    df = (
        read_seamf_zipfile(zfile, allowlist=files, trace_type='pfp')
        .astype('float32')
        .reset_index(['frequency', 'detector', 'capture_statistic'])
    )
    df.columns = df.columns.astype('str')
    
    return df

with stopwatch():
    # outline the structure of the dataframe
    col_stub = read_for_export(filelist[:1]).iloc[:1]
    index_divs = _read_seamf_zipfile_divisions(zfile, partition_size, filelist)

    ddf = dataframe.from_map(
        read_for_export,
        file_blocks,
        meta=col_stub,
        divisions=index_divs
    )

    ddf = ddf.repartition(freq='1MS')
    # df = ddf.compute(scheduler='processes')
    ddf.to_parquet('data/parquet', name_function=lambda i: ddf.divisions[i].strftime('%Y-%m-%d.parquet'))
    # df = dask.compute(ddf, scheduler='processes')[0]
    # pfp = df.compute(scheduler='processes')

 INFO   2023-02-21 18:14:04.224 • labbench: 598.037 s elapsed


In [33]:
data_path

WindowsPath('data/NIT-2022-12-13.zip')

In [34]:
from zipfile import ZipFile
import seamf

with ZipFile('data/NIT-2022-12-13.zip') as zf:
    with zf.open('NIT/20221026_NIT_ContinuousDataProduct_5282.sigmf') as fd:
        meta = seamf.read_seamf_meta(fd)

meta['captures'][0]

{'core:sample_start': 0,
 'core:frequency': 3555000000.0,
 'core:datetime': '2022-11-01T04:35:43.203Z'}

In [31]:
seamf.iso_to_datetime(meta['captures'][0]['core:datetime'], 'America/New_York')

Timestamp('2022-11-01 00:35:43.203000')

In [14]:
col_stub

,Frame time elapsed (s),capture_statistic,detector,0.0,1.7857142857142855e-05,3.571428571428571e-05,5.357142857142857e-05,7.142857142857142e-05,8.928571428571427e-05,0.00010714285714285714,0.00012499999999999998,...,0.00982142857142857,0.009839285714285714,0.009857142857142856,0.009874999999999998,0.009892857142857142,0.009910714285714285,0.009928571428571427,0.00994642857142857,0.009964285714285714,0.009982142857142856
datetime,frequency,,,,,,,,,,,,,,,,,,,,,
2022-10-26 20:47:37.094,3.555000e+09,min,rms,-88.4375,-89.25,-90.9375,-93.875,-94.375,-99.3125,-99.8125,-101.25,...,-89.75,-90.125,-89.4375,-88.9375,-88.3125,-88.1875,-87.6875,-88.375,-88.5,-88.0625


In [127]:
import dask
import dask.array
ziparchive = reload(ziparchive)
from ziparchive import read_seamf_zipfile, read_seamf_zipfile_metadata, MultiProcessingZipFile, concat_dicts
import numpy as np
from dask import dataframe

@dask.delayed
def select(data: dict, key: str, stub) -> pd.DataFrame:
    if data is None:
        return stub
    
    return data[key]

def zipfile_delayed(data_path, limit_count: int=None, partition_size=40) -> dask.delayed:
    zfile = MultiProcessingZipFile(data_path)
    filelist = [n for n in zfile.namelist() if n.endswith('.sigmf')][:limit_count]
    file_blocks = np.split(filelist, range(0,len(filelist), partition_size))

    @dask.delayed
    def read_partition(files):
        return read_seamf_zipfile(zfile, allowlist=files)

    stubs = []

    first = read_seamf_zipfile(zfile, allowlist=filelist[:1])
    stubs.append({k: first[k].iloc[:0] for k in first.keys() if isinstance(first[k], pd.DataFrame)})

    last = read_seamf_zipfile(zfile, allowlist=filelist[-1:])
    stubs.append({k: last[k].iloc[0:] for k in last.keys() if isinstance(last[k], pd.DataFrame)})

    return [read_partition(block) for block in file_blocks], stubs

def zipfile_dask_dfs(data_path, limit_count: int=None, partition_size=40) -> dask.dataframe.DataFrame:
    dicts, (first, last) = zipfile_delayed(data_path, limit_count=limit_count, partition_size=partition_size)
    ddfs = {}

    for key in ('psd', 'pfp', 'pvt', 'sweep_metadata', 'channel_metadata'):
        ddfs[key] = dataframe.from_delayed(
            [select(d, key, first[key]) for d in dicts if d is not None],
            meta=first[key]
        )

    return ddfs

partition_size = 150

with stopwatch():
    ddfs = zipfile_dask_dfs(data_path, limit_count=None, partition_size=partition_size)
    dfs, = dask.compute(ddfs, scheduler='processes')

    # day, = dask.compute(
    #     {
    #         k: ddfs[k].loc['2022-12-13']
    #         for k in ('psd', 'pfp', 'pvt')
    #     },
    #     scheduler='processes'
    # )

 INFO   2023-02-17 17:59:22.977 • labbench: 282.111 s elapsed
